## saving masks to a nucleus image dataset

In [ ]:
import h5py
import pytiff
import numpy as np
import pandas as pd
from skimage.measure import label

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

In [ ]:
!ls -lha /mnt/linux-data/codex/preprocessed_data/201021_BreastFFPE_Final/*.tif

In [ ]:
!ls -lha /mnt/linux-data/codex/preprocessed_data/201021_BreastFFPE_Final/*.csv

In [ ]:
tiff_path = '/mnt/linux-data/codex/preprocessed_data/201021_BreastFFPE_Final/201021_BreastFFPE_Final_2_nuclei.tif'
with pytiff.Tiff(tiff_path, "r") as f:
    img = f.pages[0][:]
print(img.shape)

In [ ]:
plt.matshow(img)

In [ ]:
help(label)

In [ ]:
labelimg, n_labels = label(img, neighbors=4, return_num=True)
print(labelimg.shape, n_labels)

In [ ]:
!ls tests

In [ ]:
!ls -lha weights.h5

In [ ]:
h5f = h5py.File("tests/dataset.hdf5", "r+")
print(h5f.keys())
print(h5f['cells'].keys())
print(h5f['meta'].keys())
h5f.close()

In [ ]:
use_cells = [c.decode('UTF-8') for c in h5f['meta/Cell_IDs'][:]]
print(len(use_cells))

In [ ]:
cell_table = '/mnt/linux-data/codex/preprocessed_data/201021_BreastFFPE_Final/201021_BreastFFPE_Final_3_cells.csv'
cell_df = pd.read_csv(cell_table, index_col=0, header=0)
cell_df = cell_df.loc[use_cells]
cell_df

In [ ]:
coords = np.array(cell_df.loc[:, ['X', 'Y']])
print(coords.shape)

In [ ]:
from tqdm.auto import tqdm

In [ ]:
size = 64
sizeh = int(size/2)

c = coords[0]
masks = []
for c in tqdm(coords):
    x, y = c
    bbox = [y-sizeh, y+sizeh, x-sizeh, x+sizeh]
    subimg = labelimg[bbox[0]:bbox[1], bbox[2]:bbox[3]]

    l = subimg[sizeh, sizeh]

    #plt.matshow(subimg==l)
    masks.append(subimg==l)
    
masks = np.stack(masks, axis=0)
print(masks.shape)